# Model Training
This notebook takes existing .json files from S3 and feeds them to the LLM for RAG.

In [6]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

from langchain import LLMChain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.llms import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler


In [12]:
role = 'arn:aws:iam::007715509628:role/SageMakerROle'

In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'mistralai/Mistral-7B-Instruct-v0.3',
	'SM_NUM_GPUS': json.dumps(1),
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="2.2.0"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )

In [ ]:
endpoint_name = predictor.endpoint_name
endpoint_name

# Configure LLM in LangChain

In [ ]:
model_kwargs = {"max_new_tokens": 512, "top_p": 0.8, "temperature": 0.8}

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps(
            # Mistral prompt, see https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3
            {"inputs": f"<s>[INST] {prompt} [/INST]", "parameters": {**model_kwargs}}
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        splits = response_json[0]["generated_text"].split("[/INST] ")
        return splits[0]


content_handler = ContentHandler()

In [ ]:
sm_client = boto3.client("sagemaker-runtime") # needed for AWS credentials

llm = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    model_kwargs=model_kwargs,
    content_handler=content_handler,
    client=sm_client,
)

# Zero-shot example

In [ ]:
system_prompt = """
As a helpful energy specialist, please answer the question, focusing on numerical data.
Don't invent facts. If you can't provide a factual answer, say you don't know what the answer is.
"""

prompt = PromptTemplate.from_template(system_prompt + "{content}")

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
question = "What is the trend for solar investments in China in 2023 and beyond?"

query = f"question: {question}"

In [ ]:
answer = llm_chain.run({query})
#print(answer)
answer.split('[/INST]')[1]

# RAG Example with Transcriptions

In [ ]:
# Define S3 bucket and prefix for json storage

bucket = 'aime' # default bucket for this application
prefix = "sat/???" #this would be a client-specific folder

In [ ]:
# Build list of S3 URIs

s3 = boto3.client("s3")
objs = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
objs = objs['Contents']
uris = [f's3://{bucket}/{obj["Key"]}' for obj in objs]
uris

### Analyze documents with Amazon Textract and split them in chunks

In [ ]:
%%time

textract_client = boto3.client('textract')
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)

all_chunks = []

for uri in uris:
    loader = AmazonTextractPDFLoader(uri, client=textract_client)
    document = loader.load()
    chunks = splitter.split_documents(document)
    all_chunks += chunks
    print(f"Loaded {uri}, {len(document)} pages, {len(chunks)} chunks")

### Embed document chunks and store them in FAISS

In [ ]:
# Define embedding model
# See https://huggingface.co/spaces/mteb/leaderboard

embedding_model_id = "BAAI/bge-small-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

In [ ]:
%%time
# Embed chunks
embeddings_db = FAISS.from_documents(all_chunks, embeddings)

In [ ]:
# Save database
embeddings_db.save_local("faiss_index")

### Shortcut - load existing embeddings db

In [ ]:
embeddings_db = FAISS.load_local("faiss_index", embeddings)

In [ ]:
# Define prompt template
prompt_template = """
As a helpful energy specialist, please answer the question below, focusing on numerical data and using only the context below.
Don't invent facts. If you can't provide a factual answer, say you don't know what the answer is.

question: {question}

context: {context}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever, 
    chain_type_kwargs={"prompt": prompt})

## Ask a question

In [ ]:
question = "What is the trend for solar investments in China in 2023 and beyond?"
answer = chain.run({"query": question})
#print(answer)
answer.split('[/INST]')[1]

### Configure RAG Chain

In [ ]:
retriever = embeddings_db.as_retriever(search_kwargs={"k": 10})

# SHUTDOWN CODE!!!

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()